[문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough 
- itemgetter()

세션ID 설정하고, 대화를 진행합니다. (multi-turn conversation)  
LLM 모델이 과거 대화를 알지(기억) 못하는 상황을 만드세요.

In [ ]:
## 1. 모듈 import
from operator import itemgetter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import (
    BaseChatMessageHistory, 
    InMemoryChatMessageHistory
)

## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
## type: dict 
store = {}

## 4. 함수 정의: 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 6. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 7. trim 설정
trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

## 8. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
) 

## 9. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

## 10. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '1234'}}      
    )

    print('\n' + '=' * 50 + '\n')

물론입니다! 이력서 작성을 위해 몇 가지 정보가 필요합니다. 아래 질문에 답해주시면 이력서를 작성하는 데 큰 도움이 됩니다.

1. 성명:
2. 연락처(전화번호 및 이메일):
3. 주소:
4. 경력(회사명, 직무, 근무기간):
5. 학력(학교명, 전공, 졸업년도):
6. 기술 및 능력:
7. 수상 경력 및 자격증:
8. 자기소개서 또는 목표:
9. 기타 추가하고 싶은 정보:

위 정보를 바탕으로 이력서를 작성해 드리겠습니다.

[이력서]

박보검
이메일: bogum.park@example.com | 연락처: 010-1234-5678

지원분야: 웹 개발 신입

개인 프로필
웹 개발 분야에 깊은 흥미와 열정을 가진 자체 주도형 신입 개발자입니다. 최신 기술 트렌드를 학습하고, 새로운 도전에 적극적으로 임하며, 팀과의 협업을 통해 실질적이고 혁신적인 웹 구현 솔루션을 제공하는 것을 목표로 하고 있습니다. 코드의 품질과 효율성, 그리고 사용자 경험을 최우선시하는 자세로 책임감을 가지고 프로젝트를 완수합니다.

보유 기술
- Python: 데이터 처리를 위한 스크립트를 작성하고 Django 프레임워크를 활용하여 서버 사이드 웹 개발 경험을 보유하고 있습니다.
- HTML & CSS: W3C 표준을 준수하며, 반응형 웹 디자인을 구현할 수 있는 기술을 갖추고 있습니다. 웹 접근성과 사용자 인터페이스 디자인의 최적화를 항상 고려합니다.
- LLM (Large Language Models): AI 모형을 학습하고 활용하는 데 기초적인 이해를 가지고 있으며, 향후 AI 기반 솔루션 통합을 위한 역량을 키우고 있습니다.

프로젝트 경험
1. 개인 프로젝트 – "Coffee Time" 웹사이트 구축
   - 개요: 커피 애호가 커뮤니티를 위한 정보 공유 및 레시피 게시 플랫폼 개발
   - 역할: 프런트엔드 디자인 및 백엔드 서버 구축, 사용자 가입/로그인 기능 구현
   - 기술: Python, Django, HTML, CSS

2. 학술 프로젝트 – "Chatbot for 

In [8]:
get_history('1234')

InMemoryChatMessageHistory(messages=[HumanMessage(content='이력서 작성해주세요!', additional_kwargs={}, response_metadata={}), AIMessage(content='물론입니다! 이력서 작성을 위해 몇 가지 정보가 필요합니다. 아래 질문에 답해주시면 이력서를 작성하는 데 큰 도움이 됩니다.\n\n1. 성명:\n2. 연락처(전화번호 및 이메일):\n3. 주소:\n4. 경력(회사명, 직무, 근무기간):\n5. 학력(학교명, 전공, 졸업년도):\n6. 기술 및 능력:\n7. 수상 경력 및 자격증:\n8. 자기소개서 또는 목표:\n9. 기타 추가하고 싶은 정보:\n\n위 정보를 바탕으로 이력서를 작성해 드리겠습니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44'}, id='run--a830989c-3256-4fe5-83e7-c27543c9b2dc-0'), HumanMessage(content='이름: 박보검, 지원분야: 웹 개발 신입, 보유 스킬: Python, CSS, HTML, LLM', additional_kwargs={}, response_metadata={}), AIMessage(content='[이력서]\n\n박보검\n이메일: bogum.park@example.com | 연락처: 010-1234-5678\n\n지원분야: 웹 개발 신입\n\n개인 프로필\n웹 개발 분야에 깊은 흥미와 열정을 가진 자체 주도형 신입 개발자입니다. 최신 기술 트렌드를 학습하고, 새로운 도전에 적극적으로 임하며, 팀과의 협업을 통해 실질적이고 혁신적인 웹 구현 솔루션을 제공하는 것을 목표로 하고 있습니다. 코드의 품질과 효율성, 그리고 사용자 경험을 최우선시하는 자세로 책임감을 가지고 프로젝트를 완수합

In [9]:
for info in get_history('1234').messages:
    print(info.content)
    print('*' * 50)

이력서 작성해주세요!
**************************************************
물론입니다! 이력서 작성을 위해 몇 가지 정보가 필요합니다. 아래 질문에 답해주시면 이력서를 작성하는 데 큰 도움이 됩니다.

1. 성명:
2. 연락처(전화번호 및 이메일):
3. 주소:
4. 경력(회사명, 직무, 근무기간):
5. 학력(학교명, 전공, 졸업년도):
6. 기술 및 능력:
7. 수상 경력 및 자격증:
8. 자기소개서 또는 목표:
9. 기타 추가하고 싶은 정보:

위 정보를 바탕으로 이력서를 작성해 드리겠습니다.
**************************************************
이름: 박보검, 지원분야: 웹 개발 신입, 보유 스킬: Python, CSS, HTML, LLM
**************************************************
[이력서]

박보검
이메일: bogum.park@example.com | 연락처: 010-1234-5678

지원분야: 웹 개발 신입

개인 프로필
웹 개발 분야에 깊은 흥미와 열정을 가진 자체 주도형 신입 개발자입니다. 최신 기술 트렌드를 학습하고, 새로운 도전에 적극적으로 임하며, 팀과의 협업을 통해 실질적이고 혁신적인 웹 구현 솔루션을 제공하는 것을 목표로 하고 있습니다. 코드의 품질과 효율성, 그리고 사용자 경험을 최우선시하는 자세로 책임감을 가지고 프로젝트를 완수합니다.

보유 기술
- Python: 데이터 처리를 위한 스크립트를 작성하고 Django 프레임워크를 활용하여 서버 사이드 웹 개발 경험을 보유하고 있습니다.
- HTML & CSS: W3C 표준을 준수하며, 반응형 웹 디자인을 구현할 수 있는 기술을 갖추고 있습니다. 웹 접근성과 사용자 인터페이스 디자인의 최적화를 항상 고려합니다.
- LLM (Large Language Models): AI 모형을 학습하고 활용하는 데 기초적인 이해를 가지고 있으며, 향후 AI 기반 솔루션 통합을 위